# TP1 : Logistic regression by gradient descent

## Etudiant 1 : Emilie SALEM
## Etudiant 2 : Hadrien SALEM 

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

## Question 1

In [ ]:
def datagen(nb_points):
    data = []
    labels = []
    for _ in range(nb_points):
        x1, x2 = np.random.uniform(low = 0, high = 1, size = 2)
        D = (0.5*x1 + x2 - 0.75) / np.sqrt(0.5**2 + 1)
        if D > 0 : label = 1
        else : label = 0
        r = np.exp(-D**2/(2*0.05**2))
        z = np.random.binomial(1, r/2)
        if(z==1):
            if label == 1 : label = 0
            else: label = 1
        point = np.array([x1, x2])
        data.append(point)
        labels.append(label)
    return np.array(data), np.array(labels)
    

In [ ]:
X, labels = datagen(300)

In [ ]:
class0 = X[labels == 0]
class1 = X[labels == 1]

plt.figure(figsize=(10,7))
plt.scatter(class0[:,0], class0[:,1], c='r')
plt.scatter(class1[:,0], class1[:,1], c='b')
plt.legend(["Class 0", "Class 1"])
plt.title("Distribution des points dans les classes 0 et 1")
plt.show()

## Question 2

In [ ]:
X_plus = np.hstack((np.ones((X.shape[0],1)), X))

## Question 3

In [ ]:
def sgm(x): return 1/(1+np.exp(-x))

def gradient_descent(X_plus, labels, learn_rate, iter_max):
    theta = np.random.rand(3)
    
    history = [theta]
    pred = sgm(X_plus.dot(theta)).reshape(300,)
    err = pred - labels 
    g = X_plus.T.dot(err)
    theta_new = theta - learn_rate*g
    history.append(theta_new)
    N_iter = 1
    
    while((np.linalg.norm(theta_new-theta) > 10e-8) and (N_iter < iter_max)):
        N_iter += 1
        theta = theta_new
        pred = sgm(X_plus.dot(theta)).reshape(300,)
        err = pred - labels 
        g = X_plus.T.dot(err)
        theta_new = theta - learn_rate*g
        history.append(theta_new)
        
    return history[-1], np.array(history)

## Question 4

In [ ]:
def display_descent(thetas) :
    fig = plt.figure(figsize=((7,7)))
    ax = fig.gca(projection="3d")
    ax.plot(thetas[:,0],thetas[:,1],thetas[:,2],"-o")
    plt.draw()

### Avec un learning rate de 0.02

In [ ]:
theta, thetas = gradient_descent(X_plus, labels, 0.02, 50000)
print("Nombre d'itérations : " + str(len(thetas)))
display_descent(thetas)

### Avec un learning rate de 0.1

In [ ]:
theta, thetas = gradient_descent(X_plus, labels, 0.1, 50000)
print("Nombre d'itérations : " + str(len(thetas)))
display_descent(thetas)

### Commentaire

On observe qu'avec un learning rate plus important, on fait des pas de gradient plus grands : il y a plus d'oscillations et on risque de ne pas converger, mais si on converge effectivement la solution est atteinte bien plus vite.

## Question 5

## Question 7

In [ ]:
def display_boundary(theta):
    plt.figure(figsize=(10,7))
    plt.scatter(class0[:,0], class0[:,1], c='r')
    plt.scatter(class1[:,0], class1[:,1], c='b')
    plt.title("Distribution des points dans les classes 0 et 1 + Prédiction")

    x1 = np.linspace(0,1,10) 
    x2 = (-theta[0]-theta[1]*x1)/theta[2]
    plt.plot(x1,x2,'k--')

    plt.legend(["Frontière de décision", "Class 0", "Class 1"])

    plt.show()